In [1]:
import pandas as pd
import os
import glob
from gtts import gTTS
import gradio as gr
from fuzzywuzzy import fuzz
from bing_image_downloader import downloader
import zipfile

In [2]:
## Подготовка данных
# Загружаем датасет с растениями парка Зарядье
df_plants = pd.read_csv("C:\Data_sets\data-60861-2024-08-06.csv", delimiter=';')
# Удаляем строки, где ID равен 'Код', и сбрасываем индексы
df_plants = df_plants[df_plants['ID'] != 'Код'].reset_index(drop=True)

In [3]:
# Формируем список латинских наименований растений
latin_names = df_plants['LatinName'].tolist()

print(latin_names)

['Picea abies', 'Pinus sylvestris', 'Querqus robur', 'Tilia cordata', 'Acer platanoides', 'Malus baccata', 'Acer saccharinum', 'Sorbus aucuparia', 'Padus avium', 'Betula pendula', 'Betula pubescens', 'Salix alba', 'Salix alba x blanda', 'Salix babylonica «Sverdlovskaja izvilistaja 2»', 'Juniperus communis Hornibrooki', 'Juniperus sabina Mas', 'Pinus pumila', 'Betula nana', 'Salix lanata', 'Salix glauca callicarpaea', 'Coryllus avellana', 'Euonimus verrucosus', 'Sambucus racemosa', 'Rosa canina', 'Spiraea chamaedryfolia', 'Salix ledebouriana x S. purpurea hybrida Sukaszewii', 'Salix purpurea', 'Viburnum opulus', 'Berberis vulgaris', 'Amygdalis nana', 'Cerassus fruticjsa', 'Lonicera tatarica', 'Acer tataricum', 'Syringa vulgaris', 'Rosa rugosa', 'Spiraea bumalda', 'Spiraea vanhouttei', 'Potentilla fruticosa', 'Caltha palustris', 'Carex capillaris', 'Carex capitata', 'Carex caespitosa', 'Cerastium alpinum', 'Dryas octopetala', 'Rubus arcticus', 'Polygonum viviparum', 'Saxifraga caespitosa

In [4]:
## Формируем архив с картинками растений
# Скачиваем картинки для каждого латинского наименования растения
for latin_name in latin_names:
    downloader.download(
        latin_name,
        limit=3,  # количество картинок для каждого растения
        output_dir='plant_images',
        adult_filter_off=True,
        force_replace=False,
        timeout=60
    )

# Формируем zip архив
with zipfile.ZipFile('plant_images.zip', 'w') as zipf:
    for latin_name in latin_names:
        image_dir = f'plant_images/{latin_name}'
        if os.path.exists(image_dir):
            for image in os.listdir(image_dir):
                image_path = os.path.join(image_dir, image)
                zipf.write(image_path, f'{latin_name}/{image}')

[%] Downloading Images to c:\IDE\Zaradye_bot\plant_images\Picea abies


[!!]Indexing page: 1

[%] Indexed 3 Images on Page 1.


[%] Downloading Image #1 from https://www.conifers.org/pi/pi/a/abies02.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://www.vdberk.fr/media/410420/picea_abies.jpg
[%] File Downloaded !

[%] Downloading Image #3 from http://phytoimages.siu.edu/users/paraman1/8_17_06_12/234567/PiceaAbies2.jpg
[%] File Downloaded !



[%] Done. Downloaded 3 images.
[%] Downloading Images to c:\IDE\Zaradye_bot\plant_images\Pinus sylvestris


[!!]Indexing page: 1

[%] Indexed 3 Images on Page 1.


[%] Downloading Image #1 from https://www.vdberk.com/media/414230/pinus-sylvestris.jpg
[%] File Downloaded !

[%] Downloading Image #2 from https://www.treesandshrubsonline.org/site/assets/files/6733/pinus-sylvestris.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://www.bighearttreecare.co.uk/wp-content/uploads/2019/12/Pinus-sylvestris-Scots-pine-1-scaled.jp

In [5]:
# Код для поиска растений и вывода информации по ним

def text_to_speech(text):
    """Преобразует текст в речь и возвращает путь к аудиофайлу"""
    tts = gTTS(text=text, lang='ru')  # Создаем объект gTTS с текстом и языком
    tts.save('response.mp3')  # Сохраняем аудиофайл
    return 'response.mp3'  # Возвращаем путь к аудиофайлу

def get_plant_images(latin_name):
    """Получает пути к изображениям для данного латинского названия растения"""
    image_dir = f'C:/Users/DS/Desktop/Зарядье/zaryadye_bot/plant_images/{latin_name}'  # Путь к папке с изображениями
    if os.path.exists(image_dir):  # Проверяем, существует ли папка
        return sorted(glob.glob(f'{image_dir}/Image_*.jpg'))  # Возвращаем отсортированные пути к изображениям
    return []  # Если папка не существует, возвращаем пустой список

def get_plant_info(question, speak=True):
    """Обрабатывает вопросы пользователя и возвращает информацию о растениях"""
    # Проверка на наличие запроса о хвойных растениях
    if 'хвойные' in question.lower():
        conifer_plants = df_plants[df_plants['LandscapingZone'].str.contains('Хвойный', na=False)]
        response = f"Хвойные растения в парке:\n" + "\n".join(conifer_plants['Name'].tolist())
        return response, text_to_speech(response) if speak else None, []

    # Проверка на запрос всех растений
    if 'все растения' in question.lower() or 'список растений' in question.lower():
        response = "Растения в парке:\n" + "\n".join(df_plants['Name'].tolist())
        return response, text_to_speech(response) if speak else None, []

    # Поиск растения с использованием fuzzy matching
    max_ratio = 0
    matched_name = None
    for name in df_plants['Name'].unique():
        ratio = fuzz.partial_ratio(name.lower(), question.lower())  # Вычисляем коэффициент схожести
        if ratio > max_ratio and ratio > 70:  # Проверяем, если коэффициент выше 70
            max_ratio = ratio
            matched_name = name  # Сохраняем найденное название

    # Если найдено соответствующее название растения
    if matched_name:
        plant = df_plants[df_plants['Name'] == matched_name].iloc[0]  # Получаем информацию о растении
        latin_name = plant['LatinName']  # Получаем латинское название
        images = get_plant_images(latin_name)  # Получаем изображения растения

        # Определяем ответ в зависимости от вопроса
        if any(word in question.lower() for word in ['где', 'расположен', 'растет']):
            response = f"{matched_name} расположен в {plant['LocationPlace']}"
        elif any(word in question.lower() for word in ['когда', 'цветет', 'цветение']):
            response = f"{matched_name} цветет в период: {plant['ProsperityPeriod']}"
        elif 'латинск' in question.lower():
            response = f"Латинское название {matched_name}: {plant['LatinName']}"
        else:
            response = f"{plant['Description']}"  # Возвращаем описание растения

        return response, text_to_speech(response) if speak else None, images

    # Проверка на запрос о растениях смешанного леса
    if 'смешанный лес' in question.lower():
        mixed_forest = df_plants[df_plants['LandscapingZone'].str.contains('Смешанный', na=False)]
        response = f"Растения смешанного леса:\n" + "\n".join(mixed_forest['Name'].tolist())
        return response, text_to_speech(response) if speak else None, []

    # Если вопрос не распознан
    response = "Уточните вопрос. Вы можете спросить о конкретном растении или группе растений."
    return response, text_to_speech(response) if speak else None, []

# Создаем интерфейс Gradio
iface = gr.Interface(
    fn=get_plant_info,  # Функция для обработки вопросов
    inputs=[
        gr.Textbox(lines=2, placeholder="Задайте вопрос о растениях парка"),  # Поле для ввода вопроса
        gr.Checkbox(label="Озвучить ответ", value=True)  # Чекбокс для выбора озвучивания ответа
    ],
    outputs=[
        gr.Textbox(label="Текстовый ответ"),  # Поле для текстового ответа
        gr.Audio(label="Аудио ответ"),  # Поле для аудиоответа
        gr.Gallery(label="Изображения растения")  # Галерея для отображения изображений
    ],
    title="🌳 Справочник растений парка Зарядье",  # Заголовок приложения
    description="Задавайте вопросы о растениях или группах растений. Для каждого растения доступны фотографии.",  # Описание приложения
    examples=[
        ["Какие хвойные растения есть в парке?", True],
        ["Где растет Ель?", True],
        ["Покажи все растения смешанного леса", True],
        ["Когда цветет Липа?", True],
        ["Расскажи про Дуб", True],
        ["Список всех растений", True]
    ]  # Примеры вопросов для интерфейса
)

iface.launch(share=True)  # Запускаем интерфейс и делимся ссылкой

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://ce44ec4ff999c4d263.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
